In [1]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')

import hvplot.pandas

In [2]:
weather = pd.read_csv('Test.csv')

In [3]:
weather

,date_time,is_holiday,air_pollution_index,humidity,wind_speed,wind_direction,visibility_in_miles,dew_point,temperature,rain_p_h,snow_p_h,clouds_all,weather_type,weather_description
0,18-05-2017 00:00,None,73.0,63.0,1.0,27.0,4.0,4.0,285.15,0.0,0.0,90.0,Rain,moderate rain
1,18-05-2017 00:00,None,251.0,63.0,1.0,27.0,4.0,4.0,285.15,0.0,0.0,90.0,Mist,mist
2,18-05-2017 00:00,None,75.0,56.0,1.0,0.0,1.0,1.0,285.15,0.0,0.0,90.0,Drizzle,light intensity drizzle
3,18-05-2017 01:00,None,98.0,56.0,1.0,351.0,2.0,2.0,284.79,0.0,0.0,90.0,Rain,heavy intensity rain
4,18-05-2017 01:00,None,283.0,56.0,1.0,351.0,1.0,1.0,284.79,0.0,0.0,90.0,Mist,mist
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14449,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14451,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14452,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
iweather = weather.interactive()

In [5]:
humidityslider = pn.widgets.IntSlider(name='Humidity slider', start= 25, end= 100)
humidityslider

IntSlider(end=100, name='Humidity slider', start=25, value=25)

In [6]:
# Radio buttons for air pollution index measures
yaxis_API = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['air_pollution_index',],
    button_type='success'
)

In [7]:
weatherpipeline = (
    iweather[
        (iweather.humidity <= humidityslider)
    ]
    .groupby(['humidity', 'air_pollution_index'])['visibility_in_miles'].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='humidity')  
    .reset_index(drop=True)
)

In [8]:
weatherpipeline

In [9]:
weathertable = weatherpipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size = 10, sizing_mode='stretch_width') 
weathertable

In [10]:
weatherplot = weatherpipeline.hvplot(x = 'visibility_in_miles', by='humidity', y= 'air_pollution_index',line_width=2, title="Humidity by visibility in Miles")
weatherplot

In [11]:
scatterplotpipeline = (
    iweather[
        (iweather.humidity == humidityslider)
    ]
    .groupby(['humidity', 'air_pollution_index'])['visibility_in_miles'].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='humidity')  
    .reset_index(drop=True)
)

In [12]:
scatterplotpipeline

In [13]:
weatherscatterplot = scatterplotpipeline.hvplot(x='visibility_in_miles', y='air_pollution_index', by='humidity', size=80, kind="scatter", alpha=0.7, legend=False, height=500, width=500) 

In [14]:
weatherscatterplot

In [16]:
weathersource = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['air_pollution_index'], 
    button_type='success'
)

weatherbarpipeline = (
    iweather[
        (iweather.humidity == humidityslider) 
    ]
    .groupby(['humidity', 'air_pollution_index'])['visibility_in_miles'].sum()
    .to_frame()
    .reset_index()
    .sort_values(by='humidity')  
    .reset_index(drop=True)
)

In [18]:
weatherbarplot = weatherbarpipeline.hvplot(kind='bar', x='visibility_in_miles', y='air_pollution_index', title='Visibility Vs API')

In [19]:
weatherbarplot

In [26]:
template = pn.template.FastListTemplate(
    title='Air Pollution Index VS Visibility Dashboard', 
    sidebar=[pn.pane.Markdown("# How is visibility impacted by pollution? "), 
             pn.pane.Markdown("#### On hazy days, air pollution can be visible as a plume, layered haze, or uniform haze. A plume is a column-shaped layer of air pollution coming from a point source (such as a smoke stack). Layered haze is any confined layer of pollutants that creates a contrast between that layer and either the sky or landscape behind it. Plumes and layers can mix with the surrounding atmosphere, creating a uniform haze or overall decline in air clarity. Plumes and layered haze are more common during cold winter months when the atmosphere moves less. Uniform haze occurs most often when warm air causes atmospheric pollutants to become well mixed."), 
             pn.pane.Markdown("## Settings"), humidityslider],
    main=[pn.Row(pn.Column('visibility_in_miles', weathertable.panel(width=700), margin=(0,25)), weatherplot.panel(width=500)), 
          pn.Row(pn.Column(weatherscatterplot.panel(width=600), margin=(0,25)), 
                 pn.Column('visibility_in_miles', weatherbarplot.panel(width=600)))],
    accent_base_color="#88d8b0",
    header_background="#88d8b0",
)
# template.show()
template.servable();